Zadání úloh pro seminární práci. Předpokládá se, že si student vybere jednu úlohu po konzultaci s vyučujícím.
Úlohy mají charakter výzkumných úkolů. Součástí zadání je teoretický popis řešení, úkolem je praktická realizace v Pythonu.

# Problém N těles

## Úloha

Pokuste se simulovat vývoj sluneční soustavy. Tedy pohyb velkého množství hmotných bodů, které se mohou srážet a tím spojovat. 

## Teorie

### Pohyb
Pohyb $N$ hmotných bodů s hmotnostmi $m_i$ a počátečními polohami $X_i^0$ a rychlostmi $V_i^0$ (uvažujeme pouze 2d prostor) je popsán soustavou diferenciálních rovnic druhého řádu:

$$ m_i \frac{{\rm d}^2}{{\rm d}t^2} X_i(t)= F_i $$

kde síla $F_i$ závisí na polohách ostatních těles:

$$ F_i = \sum_{j\ne i} G \frac{m_i m_j}{|R_{ij}|^3} R_{ij},\quad R_{ij}  =  X_i - X_j$$

$G$ je gravitační konstanta $G=6.674\times10^{−11}$ $m^3kg^{−1}s^{−2}$.

Zavedením rychlostí převedeme na soustavu prvního řádu:
$$ m_i \dot{V_i} = F_i$$
$$ \dot{X_i} = V_i $$

s počátečními podmínkami $X_i(0) = X^0_i$, $V_i(0)=V^0_i$ má soustava jednoznačné řešení pokud nedojde ke koincidenci bodů, což je velmi nepravděpodobné. Řešení však nelze vypočítat v uzavřeném tvaru a je proto vhodné řešení pomocí numerických metod pro soustavy diferenciálních rovnic. 

### Detekce kolize
Pro uvažování kolizí, budeme pro každé těleso uvažovat také jeho poloměr $l_i$. Kolize nastane pokud pro nějaký čas $t$ a pro nějakou dvojici $i,j$ platí:

$$ |R_{ij}| \le l_i + l_j $$.

Ovšem nemůžeme tuto podmínku testovat pro všechny časy. Budeme tedy uvažovat, že se mezi dvěma simulačními časy tělesa pohybují rovnoměrně přímočaře, tedy s počáteční pozicí $X_i$ a s rychlostí $V_i$:

$$ x_i(t) = X_i + t V_i $$

Označíme $R=R_{ij}$, $S = S_{ij} = V_i - V_j$, $l = l_{ij} = l_i + l_j$. Kolize nastane pokud:

$$ |R + t S| < l \quad\text{pro } t\in(0, \tau) $$

kde $\tau$ je aktuální časový krok. Jelikož $l$ je kladné můžeme nerovnici umocnit a levou stranu roznásobit:

$$ R.R + 2t R.S + t^2 S.S < l^2 $$.

To bude platit pokud $t$ bude v intervalu mezi kořeny $t_1$, $t_2$ rovnice:
$$ k(t) = at^2 +bt + c=0, \quad a=S.S,\ b=2R.S,\ c=R.R - l^2.$$

Pokud je $D=b^2 -4ac \le 0$ (komplexní nebo násobné kořeny) pak kolize nenastane.

Pro $D>0$ kolize nenastane pokud $T < t_1$ nebo $t_2 < 0$. Jelikož je $a\le0$, je první podmínka je ekvivalentní podmínce

$$ k(T) > 0 \text{ a } k'(T) = 2aT + b < 0$$

Druhá podmínka je ekvivalentní:

$$ k(0) = c > 0 \text{ a } k'(0) = b > 0$$.

### Průběh kolize
Při detekované kolizi vznikne ze dvou těles jedno těleso. Je třeba určit jeho polohu $X$ a rychlost $V$ v čase $T$.
Polohu určíme jako hmotný střed soustavy:
$$ X(T) = \frac{m_iX_i(T) + m_jX_j(T)}{m_i + m_j} $$

Rychlost obdobně ze zákona zachování hybnosti:
$$ V(T) = \frac{m_iV_i(T) + m_jV_j(T)}{m_i + m_j} $$


## Implementace

### Obecné poznámky
- Začněte implementací pohybu a vizualizace bez kolizí.
- Začněte s malým počtem shluků a pak ho zkuste zvyšovat.
- Zkuste udržet cílový čas simulace 100000let. Pro shluky pohybující se blízko Slunce s malými oběžnými dobami může
  řešič adaptivně zkracovat časový krok. Tomu můžete zabránit nastavením minimálního časového kroku. To může vést 
  k nestabilitě. Ovlivní tato nestabilita celý výpočet? 
- Alternativně je možné čas od času odstranit shluky, které jsou moc daleko nebo moc blízko od Slunce.
- Když bude fungovat výpočet pro dlouhé časy bez kolizí. Zkuste přidat kolize.

### Počáteční podmínky

Předpokládejte Slunce o současné hmotnosti $m_s = 333000$ $M_e$ (hmotností Země)  a poloměru $l_s = 10^{-4}AU$ $km$ ve středu souřadné soustavy. 

Tělesa o celkové průměrné hmotnosti $14000$ $M_e$ rovnomerně rozložená v mezikruží $(0.1 AU, 30AU)$ $km$. Každé bude mít hmotnost menšího asteroidu ledového asteroidu o průměru 1km,  $m = 2\times 10^{-25}$ $M_e$.

Rychlosti budou  blízko kruhovým drahám. Tj. pro těleso s pozicí $X=(X_0, X_1)$ bude směr rychlosti kolmý $N = (-X_1, X_0)$:

$$ V = \frac{\alpha N}{|X|} + E,\quad \alpha = \sqrt{\frac{Gm_s}{|X|}}$$

kde $E$ bude náhodný vektor velikostně úměrný rychlosti: $E_d = \epsilon \xi_d V_d$, $d=0,1$ a $\xi_d$ z normálního rozdělení $N(0,1)$. Faktor $\epsilon$ pak řídí rozptyl rozumnou hodnotu bude nutné vyzkoušet, asi v rozpětí $(0.001, 0.1)$.

### Výpočet 

- Neznámé polohy a rychlosti je vhodné uspořádat do jedné matice $N\times4$. 
- Z navržených hmotností těles a jejich celkové hmotnosti nám vyjde počet těles řádově $3\times 10^{29}$. To nelze   upočítat ani s efektivními algoritmy a superpočítači. Proto místo jednotlivých asteroidů budeme počítat s jejich shluky. Jejich počet bude cca $1000$ a z toho vychází jejich hmotnost $14$ $M_e$.  Průměr shluku je orientačně $1AU$.
- Při počítání se shluky se nám komplikuje výpočet kolizí. Pokud se srazí dva shluky z drobných úlomků dojde k četným
  srážkám a vznikne nový shluk se součtem hmotností, ale menším efektivním průřezem. To znamená, že pro shluky o
  poloměrech $l_i$ a $l_j$ by prostým sloučením vznikl shluk o poloměru $l = \sqrt{l_i^2 + l_j^2}$, ale výsledný shluk
  musí být ve skutečnosti menší. Vhodné asi bude použít kvadratický průmer:

  $$ l = \sqrt{\frac{l_i^2 + l_j^2}{2}}, \quad  m = m_i + m_j $$

  Při řešení soustavy může být problém se zmenšováním počtu neznámých. Je však možné výsledný shluk opět rozdělit na 2 
  menší s poloviční hmotností $\tilde m = m/2$ a poloměrem $\tilde l = l/\sqrt{2}$. Polohy nových shluků budou
  $\tilde X_i = X +/- dX$, kde $dX$ bude náhodný vektor o velikosti $l/2$. Podobně $\tilde V_i = V +/- dV$, kde $dV$ je
  náhodný vektor s velikostí úměrnou $|V_i - V_j|$.

- Dalším problémem je časová škála. Doba formování velkých planet sluneční soustavy je odhadována na 100 tisíc let. Podle třetího Keplerova zákona platí pro hlavní poloosu $a$ obíhajícího tělesa a čas oběhu $T$:
$$ \frac{a^3}{T^2} = const. $$
  pro $a=1AU$ máme $T=1y$, to je Země. Pro $a=30AU$ je $T=164y$ a naopak pro $a=0.01AU$ je $T=0.001y$. Bude tedy nutné zanedbat vše pod oběžnou drahou Země, protože i tak bude pro simulaci 100 tisíc let potřeba řádově několik miliónů časových kroků. Dalším důvodem je podmíněnost problému (resp. tuhost soustavy). Pro výrazně odlišné oběžné doby dostáváme výrazně tuhý (stiff) systém, který by bylo třeba řešit implicitní metodou, což je zde velmi nevhodné, jelikož Jakobián pravé strany je hustá matice (každé těleso je ovlivlivňováno každým).
- Předpokládá se řešení soustavy diferenciálních rovnic pomocí řešičů dostupných ve scipy. Je zde technický problém kdy volat funkci pro výpočet kolizí a jak zařídit změnu stavové proměnné (ppoloha rychlostí). Zde je nutné použít třídu `scipy.integrate.ode`, pomocí metody `integrate` nechat spočítat jen jeden krok. Vypočítat kolize a nastavit stavovou proměnnou pomocí `set_initial_value` pro aktuální čas. Zde je dokonce možné i změnit počet shluků. Je možné, že opakované nastavování počíteční podmínky povede k pomalejšímu výpočtu, zpomalení může být výraznější při zmeně počtu shluků. Případnou optimalizaci je však nutné řešit až po změření reálných časů výpočtu.

### Vizualizace

Simulaci zobrazte pomocí animovaného scatter plotu (matplotlib.animate). Prokaždá shluk zobrazte aktuální poloměr pomocí
prázdné kružnice a zobrazte hmotný střed pomocí malého plného kolečka s velikostí úměrnou třetí odmocnině hmotnosti.

Další vylepšování vizualizace dle potřeby a vaší invence.